<a href="https://colab.research.google.com/github/Munna148/ChatApp_Remainders/blob/main/Email_Remainder_time_Sensitive_Mails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plyer

In [1]:
import imaplib
import email
import smtplib
import pickle
import numpy as np
from email.mime.text import MIMEText
from email import header
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from plyer import notification
import re

# Gmail IMAP settings
EMAIL_USER = "reddymunna148@gmail.com"
EMAIL_PASS = "mvcm jgxs vdfk zjoh"  # Use an App Password if using Gmail
IMAP_SERVER = "imap.gmail.com"
IMAP_PORT = 993

# SMTP settings for sending alerts
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

# Load trained LSTM model and tokenizer
MODEL_PATH = "time_sensitive_model.h5"
TOKENIZER_PATH = "tokenizer.pkl"

try:
    model = load_model(MODEL_PATH)
    print(model.input_shape)
    with open(TOKENIZER_PATH, "rb") as handle:
        tokenizer = pickle.load(handle)
    print("✅ LSTM model and tokenizer loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model/tokenizer: {e}")
    exit()


# Function to decode MIME-encoded email subjects
def decode_subject(encoded_subject):
    decoded_parts = header.decode_header(encoded_subject)
    subject = ""
    for part, encoding in decoded_parts:
        if isinstance(part, bytes):  # Decode bytes using the specified encoding
            subject += part.decode(encoding or "utf-8", errors="ignore")
        else:
            subject += part  # Already a string
    return subject.strip()  # Remove unwanted spaces


# Function to preprocess email text
def preprocess_text(text, max_len=100):
    text = re.sub(r"\W", " ", text)  # Remove special characters
    text = text.lower().strip()

    sequences = tokenizer.texts_to_sequences([text])

    # Ensure padding to max_len = 100
    padded_seq = pad_sequences(sequences, maxlen=max_len, padding="post")

    print(f"Processed text: {text}")
    print(f"Tokenized sequence shape: {padded_seq.shape}")  # Should be (1, 100)

    return padded_seq


# Function to classify email urgency
def classify_email(subject, body):
    subject_seq = preprocess_text(subject)  # Shape: (1, 100)
    body_seq = preprocess_text(body)  # Shape: (1, 100)

    # Use only one text input (subject) and one numerical input
    num_features = np.array([[len(subject), len(body), subject.count("!") + body.count("!")]])  # Shape: (1, 3)

    print(f"Subject sequence shape: {subject_seq.shape}")  # Should be (1, 100)
    print(f"Numerical features shape: {num_features.shape}")  # Should be (1, 3)

    # Pass correct inputs to the model
    prediction = model.predict([subject_seq, num_features])[0][0]  # Use only subject_seq

    return "Time-sensitive" if prediction > 0.5 else "Normal"


# Function to send a desktop notification
def send_notification(subject, sender):
    try:
        notification.notify(
            title="🚨 Urgent Email Alert 🚨",
            message=f"From: {sender}\nSubject: {subject}",
            app_name="Email Alert",
            timeout=10
        )
        print("📢 Desktop notification sent!")
    except Exception as e:
        print(f"❌ Failed to send desktop notification: {e}")


# Function to send an email alert
def send_email_alert(subject, sender):
    email_body = f"""
    🚨 Urgent Email Received! 🚨

    📩 From: {sender}
    📌 Subject: {subject}

    Please check your inbox for details.
    """

    msg = MIMEText(email_body)
    msg["Subject"] = f"🚨 Urgent Email Alert: {subject}"  # Include received subject in alert
    msg["From"] = EMAIL_USER
    msg["To"] = EMAIL_USER  # Sending alert to yourself

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(EMAIL_USER, EMAIL_PASS)
            server.sendmail(EMAIL_USER, EMAIL_USER, msg.as_string())
        print("📧 Email alert sent successfully!")
    except Exception as e:
        print(f"❌ Failed to send email alert: {e}")


# Function to fetch latest unread emails from Gmail using IMAP
def fetch_unread_emails():
    try:
        # Connect to IMAP server
        mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
        mail.login(EMAIL_USER, EMAIL_PASS)
        mail.select("inbox")

        # Search for unread emails
        status, email_ids = mail.search(None, 'UNSEEN')
        email_ids = email_ids[0].split()

        for e_id in email_ids:
            status, msg_data = mail.fetch(e_id, "(RFC822)")
            raw_email = msg_data[0][1]
            msg = email.message_from_bytes(raw_email)

            # Extract email details
            subject = decode_subject(msg["Subject"])  # ✅ Decode subject properly
            sender = msg["From"]
            body = ""

            if msg.is_multipart():
                for part in msg.walk():
                    content_type = part.get_content_type()
                    if content_type == "text/plain":
                        try:
                            body = part.get_payload(decode=True).decode(errors="ignore")
                        except:
                            body = ""
                        break
            else:
                try:
                    body = msg.get_payload(decode=True).decode(errors="ignore")
                except:
                    body = ""

            print(f"\n📩 New Email from {sender}\n📌 Subject: {subject}")  # ✅ Properly decoded

            # Classify email urgency
            urgency = classify_email(subject, body)

            if urgency == "Time-sensitive":
                print("🚨 Urgent Email Found! 🚨")
                send_notification(subject, sender)
                send_email_alert(subject, sender)
            else:
                print("✅ Normal email detected.")

        mail.logout()

    except Exception as e:
        print(f"❌ Error fetching emails: {e}")


# Run the email checking function
fetch_unread_emails()


❌ Error loading model/tokenizer: [Errno 2] Unable to synchronously open file (unable to open file: name = 'time_sensitive_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

📩 New Email from Kotak 811 <Kotak811@kotak.in>
📌 Subject: Important: Video KYC of Kotak 811 A/c no. ending with 4860 is
 pending
❌ Error fetching emails: name 'tokenizer' is not defined


In [2]:
!pip install plyer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 5.5 MB/s eta 0:00:00


In [3]:
!pip install keras